In [1]:
import numpy as np

## Vectorized metropolis algorithm in 1D

Different types of boundary conditions to consider:
1. Periodic: ends of the lattice are connected and interact with each other.
2. Open: ends of the lattice do not interact with each other (these spins only have 1 neighbor).
3. Fixed: spin at one end of the lattice is fixed. (https://staff.fysik.su.se/~ardonne/files/theses/bachelor-thesis_christopher-litens.pdf)
4. Padded: ends padded with all up/down spins.

In [84]:

class VectorizedMetropolis1D:
    def __init__(self, beta, J, H):
        """
        Args:
            beta: 1/(k_B * T)
            J: strength of nearest neighbor interactions
            H: strength of external magnetic field
        """
        self.beta = beta
        self.J = J
        self.H = H
    
    def neighbor_coupling(self, X_lattice):
        """ 
        Args:
            X_lattice (ndarray): current state of system

        Returns:
            X_neighbors (ndarray): nearest neighbor contributions to the
            interaction energy
        """
        # Sum nearest neighbor contributions to energy by shifting X_lattice.
        # Open boundary conditions (ends of lattice do not interact with each other).
        X_neighbors = np.zeros_like(X_lattice)      # Initiate array with all 0's.
        X_neighbors[:-1] += X_lattice[1:]           # Left neighbor contribution
        X_neighbors[1:]  += X_lattice[:-1]          # Right neighbor contribution

        return X_neighbors

    
    def update_spins(self, X_lattice):
        """ 
        Args:
            X_lattice (ndarray): current state of system
        
        Returns:
            X_result (ndarray): updated state of system
        """
        # Calculate change in energy required to flip each spin.
        X_neighbors = self.neighbor_coupling(X_lattice)
        X_Ei = - self.J * X_lattice * X_neighbors - self.H * X_lattice
        X_Ef = - X_Ei
        X_deltaE = X_Ef - X_Ei

        X_result = X_lattice.copy()         # Updated matrix
        # If energy < 0 (spin is currently down), flip up.
        X_result[X_deltaE < 0] = 1

        # If energy > 0 (spin is currently up), flip down with probability
        # given by Boltzmann factor.
        X_result[(X_deltaE > 0) & (X_deltaE < np.exp(- self.beta * X_deltaE))] = -1

        return X_result


In [97]:
np.random.seed(0)
lattice = np.random.choice([-1, 1], size=5)

Ising = VectorizedMetropolis1D(beta=1, J=1, H=0.1)
# print("Initial lattice and neighbors")
print(lattice)
# print(Ising.neighbor_coupling(lattice))
# print()

# for i in range(10):
#     lattice = Ising.update_spins(lattice)
#     # print(f"Update {i}")
#     print(lattice)
#     # print(Ising.neighbor_coupling(lattice))
#     # print()

i = 0
while True:
    lattice = Ising.update_spins(lattice)
    print(lattice)
    i += 1
    if np.all(lattice == 1):
        break
print(f"{i} iterations")


[-1  1  1 -1  1]
[ 1 -1 -1  1  1]
[ 1  1  1 -1  1]
[ 1  1 -1  1  1]
[ 1 -1  1 -1  1]
[1 1 1 1 1]
5 iterations


## Nonvectorized metropolis in 1D

In [82]:

# 1d Ising using Metropolis (non-vectorized).

class SimpleMetropolis1D():

    def __init__(self, beta, J, E_B, lattice):
        self.beta = beta
        self.J = J
        self.E_B = E_B
        self.lattice = lattice

    def energy(self, i):
        current = self.lattice[i]

        # Periodic/circular boundary conditions.
        # if i == len(self.lattice) - 1:
        #     j1 = i - 1
        #     j2 = 0
        # elif i == 0:
        #     j1 = len(self.lattice) - 1
        #     j2 = i + 1
        # else:
        #     j1, j2 = i - 1, i + 1
        # neighbor1, neighbor2 = self.lattice[j1], self.lattice[j2]

        # Open boundary conditions
        if i == len(self.lattice) - 1:
            neighbor1 = self.lattice[i - 1]
            neighbor2 = None
        elif i == 0:
            neighbor1 = None
            neighbor2 = self.lattice[i + 1]
        else:
            neighbor1 = self.lattice[i - 1]
            neighbor2 = self.lattice[i + 1]
        
        m = 0
        for n in [neighbor1, neighbor2]:
            if n == 1:      # spin up
                m += 1
            else:           # spin down
                m -= 1
        
        E_up = -self.J * m - self.E_B
        E_down = self.J * m + self.E_B
        delta_E = E_down - E_up     # > 0

        if current == 1:
            return delta_E
        else:
            return -delta_E
        

    def update(self):
        i = np.random.randint(len(self.lattice))
        delta_E = self.energy(i)
        # print(i)

        if delta_E < 0:     # current is down, so flip it up.
            self.lattice[i] = 1
            return self.lattice

        # current is up, so flip it down with some probability.
        prob_down = np.exp(-self.beta * delta_E)
        p = np.random.rand()
        if p < prob_down:
            self.lattice[i] = -1
        
        return self.lattice
        

In [98]:
np.random.seed(0)
lattice = np.random.choice([-1, 1], size=5)
print(lattice)

Ising = SimpleMetropolis1D(beta=1, J=1, E_B=0.1, lattice=lattice)

# for i in range(10):
#     print(Ising.update())

i = 0
while True:
    lattice = Ising.update()
    print(lattice)
    i += 1
    if np.all(lattice == 1):
        break
print(f"{i} iterations")

[-1  1  1 -1  1]
[-1  1  1  1  1]
[-1  1  1  1  1]
[-1  1  1  1  1]
[-1  1  1  1 -1]
[ 1  1  1  1 -1]
[-1  1  1  1 -1]
[-1 -1  1  1 -1]
[-1 -1  1  1 -1]
[-1  1  1  1 -1]
[ 1  1  1  1 -1]
[ 1  1  1  1 -1]
[ 1  1  1  1 -1]
[ 1  1  1  1 -1]
[-1  1  1  1 -1]
[-1  1  1 -1 -1]
[ 1  1  1 -1 -1]
[ 1  1  1 -1 -1]
[-1  1  1 -1 -1]
[-1  1  1 -1 -1]
[-1  1  1  1 -1]
[-1  1  1  1 -1]
[-1  1  1  1 -1]
[-1  1  1  1  1]
[1 1 1 1 1]
24 iterations


## Non-vectorized heat bath in 1D

In [ ]:
""" 
(pseudocode)
1d Ising using heat bath (non-vectorized).

lattice = np.random.randint(2, size=n) 

# Energy of spin at position i.
def energy(self, i):
    current = self.lattice[i]
    neighbor1, neighbor2 = self.lattice[i-1], self.lattice[i+1]
    for n in [neighbor1, neighbor2]:
        if n == up:
            m += 1
        else:
            m -= 1

    E_up = -J * m - E_B
    E_down = J * m + E_B
    return (E_up, E_down)


def update(self):
    i = np.random.randint(len(self.lattice))
    E_up, E_down = self.energy(i)
    prob_up = exp(-self.beta * E_up) / (exp(-self.beta * E_up) + exp(-self.beta * E_down))
    p = np.random.rand()

    if p < prob_up:
        self.lattice[i] = up
    else:
        self.lattice[i] = down

"""